In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import mahalanobis

In [2]:
df = pd.read_csv('../data/allinone.csv')

### ELIMINAR COLUMNAS 

In [3]:
# Eliminar columnas con un solo valor único
df = df.loc[:, df.nunique() > 1]

In [4]:
# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

In [5]:
# Eliminar columnas no relevantes
df = df.drop(columns=['Unnamed: 0'])

### Manejo de Valores Nulos

In [6]:
# Comprobamos la cantidad de nulos en las columnas
df.isnull().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 69, dtype: int64

In [7]:
# Eliminamos las columnas con >50% de valores nulos 

umbral = len(df) * 0.5 
df = df.dropna(thresh=umbral, axis=1) 

In [8]:
# Verificar valores infinitos en columnas numéricas
inf_check = np.isinf(df.select_dtypes(include=['int64', 'float64'])).sum()
print("Valores infinitos por columna:\n", inf_check)

Valores infinitos por columna:
  Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
 Active Min                    0
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
Length: 68, dtype: int64


In [9]:
df = df.replace([np.inf, -np.inf], np.nan)  # Reemplazar inf con NaN

In [10]:
# Imputación para columnas numéricas (todas las int y float)
col_num = df.select_dtypes(include=['int64', 'float64']).columns
imputer_num = SimpleImputer(strategy='mean')
df[col_num] = imputer_num.fit_transform(df[col_num])

In [11]:
# Imputación para columnas categóricas 
col_cat = df.select_dtypes(include=['object']).columns
imputer_cat = SimpleImputer(strategy='most_frequent')
df[col_cat] = imputer_cat.fit_transform(df[col_cat])

In [12]:
# Eliminar filas con nulos residuales 

df = df.dropna()

###  CORREGIR INCONSISTENCIAS 

In [13]:
# Estandarizar formatos en columnas categóricas (texto en minúsculas) 
for i in col_cat:
    df[i] = df[i].str.lower().str.strip()

In [14]:
# Eliminar duplicados 

df=df.drop_duplicates() 

### MANEJO DE OUTLIERS 

In [15]:
# Método IQR con cuantiles para la detección de outliers (no se detectan outliers)
Q1 = df[col_num].quantile(0.25)
Q3 = df[col_num].quantile(0.75)
IQR = Q3 - Q1

# Filtramos las filas cuyos valores estén dentro de los límites
df_filtrado = df[(df[col_num] >= (Q1 - 1.5 * IQR)) & (df[col_num] <= (Q3 + 1.5 * IQR))]

print("Filas originales:", len(df))
print("Filas después del filtro IQR:", len(df_filtrado))

Filas originales: 2282168
Filas después del filtro IQR: 2282168


In [16]:
# Método de la Desviación Absoluta Mediana (MAD) (no se detectan outliers)
median = df[col_num].median()
mad = np.median(np.abs(df[col_num] - median))

# Verificar si MAD es cero para evitar división por cero
if mad == 0:
    print("MAD es 0; revisar la variabilidad de la columna")
else:
    z_score = 0.6745 * (df[col_num] - median) / mad
    
    # Filtrar las filas que no son outliers
    df_filtrado = df[(np.abs(z_score) <= 3.5)]
    
    print("Filas originales:", len(df))
    print("Filas después del filtro MAD:", len(df_filtrado))

Filas originales: 2282168
Filas después del filtro MAD: 2282168


#### Distancia de Mahalanobis

La distancia de Mahalanobis es una medida que permite calcular la distancia entre un punto y una distribución, teniendo en cuenta las correlaciones entre variables en un conjunto de datos multivariado. A diferencia de la distancia euclidiana, que trata a todas las variables por igual, la distancia de Mahalanobis ajusta las distancias según la varianza y covarianza de los datos, lo que la hace especialmente útil para detectar outliers en datos con múltiples variables interrelacionadas.

In [17]:
# Método con la distancia de Mahalanobis (no se encuentran outliers)

data = df[col_num].values

# Calcular la media y la matriz de covarianza
mean = np.mean(data, axis=0)
cov_matrix = np.cov(data, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Calcular la distancia de Mahalanobis para cada punto
mahalanobis_distances = [mahalanobis(row, mean, inv_cov_matrix) for row in data]

# Determinar un umbral para identificar outliers
threshold = np.percentile(mahalanobis_distances, 97.5)  # Por ejemplo, el percentil 97.5

# Identificar los outliers
outliers = df[mahalanobis_distances > threshold]

print("Outliers encontrados: ", outliers)

LinAlgError: Singular matrix

### NORMALIZACIÓN/ESTANDARIZACIÓN DE LOS DATOS 

In [ ]:
# Estandarización (media = 0 desviación = 1) 

# scaler = StandardScaler() 
# df[col_num] = scaler.fit_transform(df[col_num]) 

### CODIFICACIÓN DE VARIABLES CATEGÓRICAS 

In [ ]:
# Identificar primero variables categóricas 

# print(df.dtypes) 

# for col in df.select_dtypes(include=['object']).columns: 
#     		print(f"{col}: {df[col].nunique()} valores únicos") 

In [ ]:
# Codificación ordinal para categrorías con orden con el uso de LabelEncoder de la librería de Sci-kit learn
# for i in col_cat:
#     le = LabelEncoder()
#     df[i] = le.fit_transform(df[i]) 

In [ ]:
# One-Hot Enccoding para categorías sin orden 

# df = pd.get_dummies(df, columns=col_cat, drop_first=True, dummy_na=True)

### FEATURE ENGINEERING

In [18]:
# TOTAL PAQUETES
df['Total_Packets'] = df[' Total Fwd Packets'] + df[' Total Backward Packets']

In [19]:
# Ratio de paquetes enviados/recibidos para identificar desequilibrios en la comunicación
df['Fwd_Bwd_Packet_Ratio'] = df[' Total Fwd Packets'] / (df[' Total Backward Packets'] + 1)

In [20]:
df['Duration_per_Packet'] = df[' Flow Duration'] / df['Fwd_Bwd_Packet_Ratio']

In [21]:
# Tasa de bytes por segundo
df['Bytes_per_Second'] = (df['Total Length of Fwd Packets'] + df[' Total Length of Bwd Packets']) / (df[' Flow Duration'] / 1e6 + 1)

In [22]:
# Numero total de flags activos
df['Total_Active_Flags'] = (df['FIN Flag Count'] + df[' SYN Flag Count'] + df[' RST Flag Count'] + df[' PSH Flag Count'] + df[' ACK Flag Count'] + df[' URG Flag Count'] + df[' ECE Flag Count'])

### GUARDAR DATOS LIMPIOS EN CSV 

In [23]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Total_Packets,Fwd_Bwd_Packet_Ratio,Duration_per_Packet,Bytes_per_Second,Total_Active_Flags
count,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,...,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06,2.282168e+06
mean,8.691529e+03,1.719367e+07,1.061335e+01,1.200899e+01,6.050673e+02,1.926324e+04,2.378921e+02,1.932659e+01,6.516634e+01,8.026530e+01,...,6.748997e+04,9.983936e+06,6.096482e+05,1.044332e+07,9.508552e+06,2.262234e+01,1.157319e+00,1.833548e+07,1.303366e+03,7.944661e-01
std,1.908598e+04,3.568801e+07,8.344993e+02,1.110099e+03,6.164660e+03,2.519535e+06,7.796726e+02,6.297020e+01,2.011317e+02,3.080722e+02,...,6.228916e+05,2.575515e+07,5.087520e+06,2.655031e+07,2.548740e+07,1.944185e+03,4.053646e+00,4.101946e+07,3.245320e+04,6.942686e-01
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,8.000000e-03,-2.600000e+01,0.000000e+00,0.000000e+00
25%,5.300000e+01,2.190000e+02,2.000000e+00,1.000000e+00,2.300000e+01,6.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.000000e-01,3.075000e+02,1.199927e+01,0.000000e+00
50%,8.000000e+01,5.882250e+04,2.000000e+00,2.000000e+00,6.700000e+01,1.600000e+02,4.100000e+01,2.000000e+00,3.700000e+01,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,6.666667e-01,8.721300e+04,1.382544e+02,1.000000e+00
75%,4.430000e+02,5.528671e+06,6.000000e+00,5.000000e+00,3.410000e+02,1.514000e+03,2.170000e+02,3.700000e+01,5.271429e+01,7.825611e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e+01,1.200000e+00,3.897812e+06,3.083126e+02,1.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,2.428415e+06,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.125597e+03,...,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08,5.116810e+05,5.460000e+02,1.474397e+10,1.095775e+07,3.000000e+00


In [24]:
df.shape

(2282168, 74)

In [25]:
df.to_csv('../data/datos_limpios.csv')